In [1]:
from divisiveclustering.datautils import DataUtils
from divisiveclustering.coresetsUtils import Coreset, normalize_np ,gen_coreset_graph
from divisiveclustering.vqe_utils import kernel_two_local, create_Hamiltonian_for_K2, get_Hamil_variables
from divisiveclustering.quantumutils import get_probs_table
import cudaq
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * number_of_qubits 


In [3]:
data_utils = DataUtils('../../data')

In [4]:
try:
    raw_data = data_utils.load_dataset()
except:
    raw_data = data_utils.create_dataset(n_samples = 1000)
    


Data loaded from ../../data/dataset.pickle


In [5]:
coresets = Coreset()

In [6]:
coreset_vectors, coreset_weights = coresets.get_coresets(
    data_vectors=raw_data, number_of_runs=10, coreset_numbers=5, size_vec_list=10
)

In [7]:
best_coreset_vectors, best_coreset_weights = coresets.get_best_coresets(raw_data, coreset_vectors, coreset_weights)

In [8]:
normalized_cv = normalize_np(best_coreset_vectors, centralize=True)
normalized_cw = normalize_np(best_coreset_weights, centralize=False)

In [9]:
coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(normalized_cv, normalized_cw, metric="dot")

In [10]:
H = create_Hamiltonian_for_K2(G, number_of_qubits, weights)

In [46]:
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)

optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(number_of_qubits, layer_count),
    spin_operator=H,
    optimizer=optimizer,
    parameter_count=parameter_count,
    shots = 1000)

counts = cudaq.sample(kernel_two_local(number_of_qubits, layer_count), optimal_parameters, shots_count = 100000)

print(counts)

<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.381290
<H> = -1.187282
<H> = -1.187282
<H> = -0.273441
<H> = -1.183502
<H> = -0.742785
<H> = -1.183502
<H> = -1.192514
<H> = -1.163320
<H> = -0.744175
<H> = -1.163320
<H> = -0.711615
<H> = -1.191931
<H> = -0.809453
<H> = -1.191931
<H> = -1.202623
<H> = -1.201245
<H> = -0.623010
<H> = -1.201245
<H> = -0.786139
<H> = -1.202859
<H> = -1.069231
<H> = -0.724278
<H> = -0.810091
<H> = -1.202859
<H> = -1.198896
<H> = -1.202781
<H> = -1.023313
<H> = -1.202658
<H> = -1.002703
<H> = -1.203090
<H> = -1.176892
<H> = -1.203371
<H> = -1.080313
<H> = -1.209102
<H> = -1.172763
<H> = -1.210124
<H> = -1.159076
<H> = -1.210943
<H> = -1.207096
<H> = -1.212733
<H> = -0.928445
<H> = -1.213124
<H> = -1.026084
<H> = -1.213139
<H> = -1.059286
<H> = -1

In [47]:
probs_table = get_probs_table(counts, sort_values= True)
best_bitstring = probs_table.iloc[0][0]
print(f"best string is {best_bitstring}")

best string is 01100


In [48]:
probs_table

,bitstring,probability
0,01100,0.67187
1,10011,0.32812
2,11111,0.00001


In [49]:
[bitstring for bitstring in counts.get_sequential_data()]

['01100', '10011', '11111']

In [50]:
solutions = [bin(i) for i in range(2**5)]

In [51]:
solutions

['0b0',
 '0b1',
 '0b10',
 '0b11',
 '0b100',
 '0b101',
 '0b110',
 '0b111',
 '0b1000',
 '0b1001',
 '0b1010',
 '0b1011',
 '0b1100',
 '0b1101',
 '0b1110',
 '0b1111',
 '0b10000',
 '0b10001',
 '0b10010',
 '0b10011',
 '0b10100',
 '0b10101',
 '0b10110',
 '0b10111',
 '0b11000',
 '0b11001',
 '0b11010',
 '0b11011',
 '0b11100',
 '0b11101',
 '0b11110',
 '0b11111']

In [52]:
solution_binary = []
qubits = 5

for solution in solutions:
    str_val = str(solution)
    str_val = str_val.split("b")[1]
    if len(str_val) < qubits:
        number_of_zeros_to_add = qubits - len(str_val)
        str_val = "0" * number_of_zeros_to_add + str_val
    solution_binary.append(str_val)

In [53]:


solution_binary

['00000',
 '00001',
 '00010',
 '00011',
 '00100',
 '00101',
 '00110',
 '00111',
 '01000',
 '01001',
 '01010',
 '01011',
 '01100',
 '01101',
 '01110',
 '01111',
 '10000',
 '10001',
 '10010',
 '10011',
 '10100',
 '10101',
 '10110',
 '10111',
 '11000',
 '11001',
 '11010',
 '11011',
 '11100',
 '11101',
 '11110',
 '11111']

In [54]:
import pandas as pd
df = pd.DataFrame(columns=['bitstring', 'probability'])


for bitstring in solution_binary:
    df.loc[len(df)] = [bitstring, counts.probability(bitstring)]

In [55]:
df

,bitstring,probability
0,00000,0.00000
1,00001,0.00000
2,00010,0.00000
3,00011,0.00000
4,00100,0.00000
5,00101,0.00000
6,00110,0.00000
7,00111,0.00000
8,01000,0.00000
9,01001,0.00000
